<a href="https://colab.research.google.com/github/gurovic/MLCourse/blob/main/012_base_stat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Подготовка среды

In [ ]:
!pip install pandas numpy matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Создание демо-данных

In [ ]:
data = {
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9],
    'temperature': [26.6, 27.2, 25.8, 18.1, 26.9, 12.4, 20.2, 30.0, 22.1],
    'weather': ['солнечно', 'дождь', 'облачно', 'солнечно', 'дождь', 'солнечно', 'облачно', 'солнечно','облачно', ],
    'timestamp': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-10-04', '2024-01-04', '2023-02-04', '2023-01-05'],
    'city': ['Москва', 'Екатеринбург', 'Екатеринбург', 'Санкт-Петербург', 'Казань', 'Санкт-Петербург', 'Сочи', 'Сочи', 'Екатеринбург'],
    'feedback': ['Отличный сервис!', 'Плохая погода...', 'OK:)', 'Нормально', '', 'Все понравилось', 'Не впечатлило', 'Прекрасное место', 'Удовлетворительно']
}

df = pd.DataFrame(data)
df.to_csv('multitype_data.csv', index=False)
df.head()

## 🟢 Базовый уровень

### 1.1 Основные типы данных

In [ ]:
print("Типы столбцов:")
print(df.dtypes)

print("\nЧисловые: int64, float64 (непрерывность, масштаб)")
print("Категориальные: object (дискретность, ограниченные значения)")
print("Временные: object (цикличность, порядок)")
print("Гео/текст: object (пространственные/семантические связи)")

### 1.2 Преобразование базовых типов

In [ ]:
# Числовой -> float (проверка масштаба)
df['temperature'] = pd.to_numeric(df['temperature'], errors='coerce')

# Временной -> datetime (обязательная конвертация)
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Категориальный -> category (анализ уникальных значений)
df['weather'] = df['weather'].astype('category')
print(f"Уникальные значения погоды: {df['weather'].cat.categories.tolist()}")

### 1.3 Специфичные операции по типам

In [ ]:
# Числовые: анализ распределения
print(f"Распределение температуры:\n{df['temperature'].describe()}")
df['temperature'].plot(kind='hist', title='Распределение температуры')
plt.show()

# Категориальные: анализ частот
print(f"Частоты погодных условий:\n{df['weather'].value_counts()}")
df['weather'].value_counts().plot(kind='bar', color='skyblue')
plt.show()

# Временные: проверка хронологии
print(f"Диапазон дат: {df['timestamp'].min()} - {df['timestamp'].max()}")

## 🟡 Продвинутый уровень

### 2.1 Особенности работы с типами

| Тип          | Ключевые характеристики              | Важные операции               |
|--------------|--------------------------------------|--------------------------------|
| **Числовой** | Масштаб, распределение, выбросы     | `describe()`, `skew()`, `kurtosis()` |
| **Категориальный** | Дисбаланс классов, ограниченный набор | `nunique()`, `value_counts()` |
| **Временной** | Частота, тренды, сезонность         | `dt` аксессор, `diff()`       |
| **Гео**      | Пространственные кластеры           | Группировка (`groupby`)        |
| **Текст**    | Длина, ключевые слова               | `str.len()`, `str.contains()`  |

### 2.2 Глубокий анализ по типам

In [ ]:
# Числовые: проверка асимметрии
print(f"Асимметрия температуры: {df['temperature'].skew():.2f}")

# Категориальные: выявление дисбаланса
weather_counts = df['weather'].value_counts(normalize=True)
print(f"Дисбаланс классов: {weather_counts.to_dict()}")

# Текстовые: базовая очистка
df['feedback_clean'] = df['feedback'].str.replace(r'[^\w\s]', '', regex=True)
print("\nОчищенные отзывы:")
print(df['feedback_clean'])

### 2.3 Группировка по типам

In [ ]:
# Группировка по городу + температурные статистики
city_stats = df.groupby('city')['temperature'].agg(['mean', 'std', 'count'])
print("Статистики по городам:")
display(city_stats)

# Анализ временных паттернов
df['day_of_week'] = df['timestamp'].dt.day_name()
print("Температура по дням недели:")
display(df.groupby('day_of_week')['temperature'].mean())

In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=city_stats)

## 🔴 Экспертный уровень

### 3.1 Автоматизация обработки типов

In [ ]:
def auto_handle_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """Автоматическое преобразование типов с сохранением семантики"""
    for col in df.columns:
        # Числовые данные
        if pd.api.types.is_numeric_dtype(df[col]):
            # Проверка масштаба
            if df[col].min() > 0 and df[col].max() / df[col].min() > 1000:
                print(f"⚠️ Большой масштаб в колонке {col}!")

        # Текстовые данные
        elif pd.api.types.is_string_dtype(df[col]):
            # Автоопределение категорий
            if df[col].nunique() < 20:
                df[col] = df[col].astype('category')

        # Временные данные
        elif pd.api.types.is_datetime64_any_dtype(df[col]):
            # Проверка частоты
            freq = df[col].diff().mode()[0]
            print(f"Частота данных в {col}: {freq}")

    return df

df = auto_handle_dtypes(df.copy())
print("\nТипы данных после автоматической обработки:")
print(df.dtypes)

### 3.2 Семантический анализ типов

In [ ]:
from pandas.api.types import infer_dtype

type_mapping = {
    'integer': 'числовой',
    'floating': 'числовой',
    'categorical': 'категориальный',
    'datetime': 'временной',
    'string': 'текстовый'
}

print("Семантические типы данных:")
for col in df.columns:
    semantic_type = infer_dtype(df[col])
    print(f"{col}: {type_mapping.get(semantic_type, 'специальный')}")

## 📊 Чеклист

| Уровень  | Навыки |
|----------|--------|
| 🟢       | Определение типов, базовое преобразование, анализ распределений |
| 🟡       | Группировка по типам, выявление дисбаланса, временные паттерны |
| 🔴       | Автоматическая обработка, семантический анализ, масштабирование |

## ⚠️ Критические ошибки

1. **Путаница типов**: обработка ID-номеров как числовых данных  
2. **Игнорирование масштаба**: смешение признаков в разных диапазонах (возраст и доход)  
3. **Некорректная группировка**: объединение редких категорий без анализа

## 💡 Ключевые принципы обработки

1. **Числовые данные**:  
   - Всегда анализируйте распределение (`df.describe()`, `.skew()`)  
   - Проверяйте масштаб: признаки в разных диапазонах требуют нормализации  
   - Ищите выбросы: они искажают статистические показатели

2. **Категориальные данные**:  
   - Определяйте уникальные значения: `df[col].nunique()`  
   - Анализируйте дисбаланс классов: `value_counts(normalize=True)`  
   - Для высококардинальных признаков (>100 категорий) используйте частотные методы

3. **Временные данные**:  
   - Немедленная конвертация в `datetime` после загрузки  
   - Проверяйте полноту временного ряда: `df['timestamp'].diff().mean()`  
   - Извлекайте цикличные компоненты (день недели, месяц)

4. **Геоданные**:  
   - На начальном этапе обрабатывайте как категориальные  
   - Группируйте по регионам для выявления пространственных паттернов  
   - Анализируйте распределение объектов: `df['city'].value_counts()`

5. **Текстовые данные**:  
   - Начинайте с базовой очистки: удаление спецсимволов  
   - Анализируйте длину текста: `df['text'].str.len().describe()`  
   - Выявляйте ключевые слова: `df['text'].str.contains('проблема').sum()`

## 📌 Итог

Тип данных определяет стратегию анализа:
- **Числовые** → Распределение и масштаб
- **Категориальные** → Частоты и дисбаланс
- **Временные** → Хронология и цикличность
- **Гео/текст** → Группировка и частотный анализ

> 💡 Золотое правило: "Правильная идентификация типа данных - фундамент для всех последующих аналитических и ML операций. Ошибка на этом этапе исказит все результаты."